# Praktische Einführung in Unsupervised Learning

| Variable Name | Role      | Type        | Description                                                                                          | Units    | Missing Values |
|---------------|-----------|-------------|------------------------------------------------------------------------------------------------------|----------|----------------|
| InvoiceNo     | ID        | Categorical | a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation |          | no             |
| StockCode     | ID        | Categorical | a 5-digit integral number uniquely assigned to each distinct product                                 |          | no             |
| Description   | Feature   | Categorical | product name                                                                                         |          | no             |
| Quantity      | Feature   | Integer     | the quantities of each product (item) per transaction                                                |          | no             |
| InvoiceDate   | Feature   | Date        | the day and time when each transaction was generated                                                 |          | no             |
| UnitPrice     | Feature   | Continuous  | product price per unit                                                                               | sterling | no             |
| CustomerID    | Feature   | Categorical | a 5-digit integral number uniquely assigned to each customer                                         |          | no             |
| Country       | Feature   | Categorical | the name of the country where each customer resides                                                  |          | no             |

In [92]:
import pandas as pd
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

In [64]:
df = pd.read_excel("data/online_retail_II 2.xlsx", sheet_name=0)
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [65]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [66]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,525461.000000,525461,525461.000000,417534.000000
mean,10.337667,2010-06-28 11:37:36.845017856,4.688834,15360.645478
min,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000
25%,1.000000,2010-03-21 12:20:00,1.250000,13983.000000
50%,3.000000,2010-07-06 09:51:00,2.100000,15311.000000
75%,10.000000,2010-10-15 12:45:00,4.210000,16799.000000
max,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000
std,107.424110,NaN,146.126914,1680.811316


In [67]:
filter = df["Customer ID"].isna()
df[filter]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
525231,538159,21324,NaN,-18,2010-12-09 17:17:00,0.00,NaN,United Kingdom
525232,538158,20892,NaN,-32,2010-12-09 17:17:00,0.00,NaN,United Kingdom
525233,538160,20956,NaN,288,2010-12-09 17:18:00,0.00,NaN,United Kingdom
525234,538161,46000S,Dotcom sales,-100,2010-12-09 17:25:00,0.00,NaN,United Kingdom


In [68]:
filter = df["Quantity"] < 0
df[filter]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
525231,538159,21324,NaN,-18,2010-12-09 17:17:00,0.00,NaN,United Kingdom
525232,538158,20892,NaN,-32,2010-12-09 17:17:00,0.00,NaN,United Kingdom
525234,538161,46000S,Dotcom sales,-100,2010-12-09 17:25:00,0.00,NaN,United Kingdom
525235,538162,46000M,Dotcom sales,-100,2010-12-09 17:25:00,0.00,NaN,United Kingdom


In [69]:
df["Invoice"] = df["Invoice"].astype("str")
# Die Zeichenkette beginnt mit genau 6 Ziffern:
filter = df["Invoice"].str.match(r"\d{6}") == False
df[filter]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.10,12605.0,Germany
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.50,15329.0,United Kingdom
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.95,15329.0,United Kingdom
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.75,15329.0,United Kingdom


In [70]:
df["Invoice"].str.replace("[0-9]", "", regex=True).unique()

array(['', 'C', 'A'], dtype=object)

In [71]:
filter = df["Invoice"].str.startswith("A")
df[filter]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom


In [72]:
filter = df["StockCode"].str.match(r"^\d{5}$") == False
df[filter]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
12,489436,48173C,DOOR MAT BLACK FLOCK,10,2009-12-01 09:06:00,5.95,13078.0,United Kingdom
23,489436,35004B,SET OF 3 BLACK FLYING DUCKS,12,2009-12-01 09:06:00,4.65,13078.0,United Kingdom
28,489436,84596F,SMALL MARSHMALLOWS PINK BOWL,8,2009-12-01 09:06:00,1.25,13078.0,United Kingdom
...,...,...,...,...,...,...,...,...
525387,538170,84029E,RED WOOLLY HOTTIE WHITE HEART.,2,2010-12-09 19:32:00,3.75,13969.0,United Kingdom
525388,538170,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2,2010-12-09 19:32:00,3.75,13969.0,United Kingdom
525389,538170,85232B,SET OF 3 BABUSHKA STACKING TINS,2,2010-12-09 19:32:00,4.95,13969.0,United Kingdom
525435,538171,47591D,PINK FAIRY CAKE CHILDRENS APRON,1,2010-12-09 20:01:00,1.95,17530.0,United Kingdom


In [73]:
filter = (df["StockCode"].str.match(r"^\d{5}$") == False) & (df["StockCode"].str.match(r"^\d{5}[a-zA-Z]+$") == False)
df[filter]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.00,12682.0,France
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.00,12636.0,USA
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.00,12362.0,Belgium
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.00,12533.0,Germany
735,C489535,D,Discount,-1,2009-12-01 12:11:00,9.00,15299.0,United Kingdom
...,...,...,...,...,...,...,...,...
524776,538147,M,Manual,1,2010-12-09 16:11:00,15.00,13090.0,United Kingdom
524887,538148,DOT,DOTCOM POSTAGE,1,2010-12-09 16:26:00,547.32,NaN,United Kingdom
525000,538149,DOT,DOTCOM POSTAGE,1,2010-12-09 16:27:00,620.68,NaN,United Kingdom
525126,538153,DOT,DOTCOM POSTAGE,1,2010-12-09 16:31:00,822.94,NaN,United Kingdom


In [74]:
df[filter]["StockCode"].unique()

array(['POST', 'D', 'DCGS0058', 'DCGS0068', 'DOT', 'M', 'DCGS0004',
       'DCGS0076', 'C2', 'BANK CHARGES', 'DCGS0003', 'TEST001',
       'gift_0001_80', 'DCGS0072', 'gift_0001_20', 'DCGS0044', 'TEST002',
       'gift_0001_10', 'gift_0001_50', 'DCGS0066N', 'gift_0001_30',
       'PADS', 'ADJUST', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_90', 'DCGSSGIRL', 'DCGS0006', 'DCGS0016', 'DCGS0027',
       'DCGS0036', 'DCGS0039', 'DCGS0060', 'DCGS0056', 'DCGS0059', 'GIFT',
       'DCGSLBOY', 'm', 'DCGS0053', 'DCGS0062', 'DCGS0037', 'DCGSSBOY',
       'DCGSLGIRL', 'S', 'DCGS0069', 'DCGS0070', 'DCGS0075', 'B',
       'DCGS0041', 'ADJUST2', '47503J ', 'C3', 'SP1002', 'AMAZONFEE'],
      dtype=object)

In [75]:
filter = df["StockCode"].str.contains(r"^DOT", na=False)
df[filter]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2379,489597,DOT,DOTCOM POSTAGE,1,2009-12-01 14:28:00,647.19,NaN,United Kingdom
2539,489600,DOT,DOTCOM POSTAGE,1,2009-12-01 14:43:00,55.96,NaN,United Kingdom
2551,489601,DOT,DOTCOM POSTAGE,1,2009-12-01 14:44:00,68.39,NaN,United Kingdom
2571,489602,DOT,DOTCOM POSTAGE,1,2009-12-01 14:45:00,59.35,NaN,United Kingdom
2619,489603,DOT,DOTCOM POSTAGE,1,2009-12-01 14:46:00,42.39,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
524272,538071,DOT,DOTCOM POSTAGE,1,2010-12-09 14:09:00,885.94,NaN,United Kingdom
524887,538148,DOT,DOTCOM POSTAGE,1,2010-12-09 16:26:00,547.32,NaN,United Kingdom
525000,538149,DOT,DOTCOM POSTAGE,1,2010-12-09 16:27:00,620.68,NaN,United Kingdom
525126,538153,DOT,DOTCOM POSTAGE,1,2010-12-09 16:31:00,822.94,NaN,United Kingdom


| Code        | Description                                                                 | Action                  |
|-------------|-----------------------------------------------------------------------------|-------------------------|
| DCGS        | Looks valid, some quantities are negative though and customer ID is null    | Exclude from clustering |
| D           | Looks valid, represents discount values                                     | Exclude from clustering |
| DOT         | Looks valid, represents postage charges                                     | Exclude from clustering |
| M or m      | Looks valid, represents manual transactions                                 | Exclude from clustering |
| C2          | Carriage transaction - not sure what this means                             | Exclude from clustering |
| C3          | Not sure, only 1 transaction                                                | Exclude                 |
| BANK CHARGES or B | Bank charges                                                          | Exclude from clustering |
| S           | Samples sent to customer                                                    | Exclude from clustering |
| TESTXXX     | Testing data, not valid                                                     | Exclude from clustering |
| gift__XXX   | Purchases with gift cards, might be interesting for another analysis, but no customer data | Exclude |
| PADS        | Looks like a legit stock code for padding                                   | Include                 |
| SP1002      | Looks like a special request item, only 2 transactions, 3 look legit, 1 has 0 pricing | Exclude for now |
| AMAZONFEE   | Looks like fees for Amazon shipping or something                            | Exclude for now          |
| ADJUSTX     | Looks like manual account adjustments by admins                             | Exclude for now          |

## 2. Datenbereinigung

In [76]:
cleaned_df = df.copy()

In [77]:
cleaned_df["Invoice"] = cleaned_df["Invoice"].astype("str")
filter = cleaned_df["Invoice"].str.match(r"^\d{6}$") == True
cleaned_df = cleaned_df[filter]
cleaned_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [78]:
cleaned_df["StockCode"] = cleaned_df["StockCode"].astype("str")
filter = (
    (cleaned_df["StockCode"].str.match(r"^\d{5}$", na=False) == True) |
    (cleaned_df["StockCode"].str.match(r"^\d{5}[a-zA-Z]+$", na=False) == True) |
    (cleaned_df["StockCode"].str.match(r"^PADS$", na=False) == True)
)

cleaned_df = cleaned_df[filter]
cleaned_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [79]:
cleaned_df = cleaned_df.dropna(subset=["Customer ID"])
cleaned_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [80]:
cleaned_df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,406337.000000,406337,406337.000000,406337.000000
mean,13.619099,2010-07-01 10:11:06.543288320,2.991462,15373.632992
min,1.000000,2009-12-01 07:45:00,0.000000,12346.000000
25%,2.000000,2010-03-26 14:01:00,1.250000,14004.000000
50%,5.000000,2010-07-09 15:48:00,1.950000,15326.000000
75%,12.000000,2010-10-14 17:09:00,3.750000,16814.000000
max,19152.000000,2010-12-09 20:01:00,295.000000,18287.000000
std,97.000659,NaN,4.285876,1677.365907


In [81]:
filter = cleaned_df["Price"] == 0
len(cleaned_df[filter])

28

In [82]:
filter = cleaned_df["Price"] > 0
cleaned_df = cleaned_df[filter]
cleaned_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [83]:
cleaned_df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,406309.000000,406309,406309.000000,406309.000000
mean,13.617924,2010-07-01 10:14:25.869572352,2.991668,15373.722268
min,1.000000,2009-12-01 07:45:00,0.001000,12346.000000
25%,2.000000,2010-03-26 14:01:00,1.250000,14006.000000
50%,5.000000,2010-07-09 15:48:00,1.950000,15326.000000
75%,12.000000,2010-10-14 17:09:00,3.750000,16814.000000
max,19152.000000,2010-12-09 20:01:00,295.000000,18287.000000
std,96.998833,NaN,4.285951,1677.329470


## 3. Feature Engineering

In [85]:
cleaned_df = cleaned_df.copy()
cleaned_df.loc[:, "SalesLineTotal"] = cleaned_df["Quantity"] * cleaned_df["Price"]
cleaned_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,SalesLineTotal
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,5.90
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,7.50


In [87]:
aggregated_df = cleaned_df.groupby(by="Customer ID", as_index=False).agg(
    MonetaryValue=("SalesLineTotal", "sum"),
    Frequency=("Invoice", "nunique"),
    lastInvoiceDate=("InvoiceDate", "max")
)
aggregated_df

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate
0,12346.0,169.36,2,2010-06-28 13:53:00
1,12347.0,1323.32,2,2010-12-07 14:57:00
2,12348.0,221.16,1,2010-09-27 14:59:00
3,12349.0,2221.14,2,2010-10-28 08:23:00
4,12351.0,300.93,1,2010-11-29 15:23:00
...,...,...,...,...
4280,18283.0,641.77,6,2010-11-22 15:30:00
4281,18284.0,411.68,1,2010-10-04 11:33:00
4282,18285.0,377.00,1,2010-02-17 10:24:00
4283,18286.0,1246.43,2,2010-08-20 11:57:00


In [91]:
max_invoice_date = aggregated_df["lastInvoiceDate"].max()
aggregated_df["Recency"] = (max_invoice_date - aggregated_df["lastInvoiceDate"]).dt.days
aggregated_df

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency
0,12346.0,169.36,2,2010-06-28 13:53:00,164
1,12347.0,1323.32,2,2010-12-07 14:57:00,2
2,12348.0,221.16,1,2010-09-27 14:59:00,73
3,12349.0,2221.14,2,2010-10-28 08:23:00,42
4,12351.0,300.93,1,2010-11-29 15:23:00,10
...,...,...,...,...,...
4280,18283.0,641.77,6,2010-11-22 15:30:00,17
4281,18284.0,411.68,1,2010-10-04 11:33:00,66
4282,18285.0,377.00,1,2010-02-17 10:24:00,295
4283,18286.0,1246.43,2,2010-08-20 11:57:00,111


In [99]:
fig1 = make_subplots(rows=1, cols=3, subplot_titles=(
    "Monentary Value Distrubution",
    "Frequency Distrubution",
    "Recency Distrubution"
))

fig1.add_trace(
    go.Histogram(x=aggregated_df["MonetaryValue"], nbinsx=10),
    row=1, col=1
)

fig1.add_trace(
    go.Histogram(x=aggregated_df["Frequency"], nbinsx=10),
    row=1, col= 2
)

fig1.add_trace(
    go.Histogram(x=aggregated_df["Recency"], nbinsx=10),
    row=1, col= 3
)

fig1.update_xaxes(title_text = "MonetaryValue", row=1, col=1)
fig1.update_xaxes(title_text = "Frequency", row=1, col=2)
fig1.update_xaxes(title_text = "Recency", row=1, col=3)

fig1.update_yaxes(title_text = "Count", row=1, col=1)
fig1.update_yaxes(title_text = "Count", row=1, col=2)
fig1.update_yaxes(title_text = "Count", row=1, col=3)

fig1.update_layout(
    height=400,
    width=1100,
    showlegend=False
)

fig1

In [104]:
fig2 = make_subplots(rows=1, cols=3, subplot_titles=(
    "Monentary Value Boxplot",
    "Frequency Boxplot",
    "Recency Boxplot"
))

fig2.add_trace(
    go.Box(y=aggregated_df["MonetaryValue"], boxpoints="outliers", marker_color="skyblue"),
    row=1, col=1
)

fig2.add_trace(
    go.Box(y=aggregated_df["Frequency"], boxpoints="outliers", marker_color="lightgreen"),
    row=1, col=2
)

fig2.add_trace(
    go.Box(y=aggregated_df["Recency"], boxpoints="outliers", marker_color="salmon"),
    row=1, col=3
)

fig2.update_layout(showlegend=False)

fig2

In [105]:
M_Q1 = aggregated_df["MonetaryValue"].quantile(0.25)
M_Q3 = aggregated_df["MonetaryValue"].quantile(0.75)
M_IQR = M_Q3 - M_Q1

filter = (aggregated_df["MonetaryValue"] > (M_Q3 + 1.5 * M_IQR)) | (aggregated_df["MonetaryValue"] < (M_Q1 - 1.5 * M_IQR))
monetary_outliers_df = aggregated_df[filter].copy()
monetary_outliers_df

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency
9,12357.0,11229.99,1,2010-11-16 10:05:00,23
25,12380.0,4782.84,4,2010-08-31 14:54:00,100
42,12409.0,12346.62,4,2010-10-15 10:24:00,55
48,12415.0,19468.84,4,2010-11-29 15:07:00,10
61,12431.0,4145.52,11,2010-12-01 10:03:00,8
...,...,...,...,...,...
4236,18225.0,7545.14,15,2010-12-09 15:46:00,0
4237,18226.0,6650.83,15,2010-11-26 15:51:00,13
4241,18231.0,4791.80,23,2010-10-29 14:17:00,41
4253,18251.0,21964.14,8,2010-09-20 13:26:00,80


In [106]:
monetary_outliers_df.describe()

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency
count,423.000000,423.000000,423.000000,423,423.000000
mean,15103.040189,12188.101740,17.167849,2010-11-09 12:26:02.978723328,30.035461
min,12357.000000,3802.040000,1.000000,2009-12-10 18:03:00,0.000000
25%,13622.000000,4605.940000,8.000000,2010-11-08 13:17:30,3.000000
50%,14961.000000,6191.320000,12.000000,2010-11-26 12:19:00,13.000000
75%,16692.000000,10273.235000,18.000000,2010-12-06 10:34:30,31.000000
max,18260.000000,349164.350000,183.000000,2010-12-09 19:32:00,364.000000
std,1728.661406,25830.851045,19.729755,NaN,51.541004


In [109]:
F_Q1 = aggregated_df["Frequency"].quantile(0.25)
F_Q3 = aggregated_df["Frequency"].quantile(0.75)
F_IQR = F_Q3 - F_Q1

frequency_outliers_df = aggregated_df[(aggregated_df["Frequency"] > (F_Q3 + 1.5 * F_IQR)) | (aggregated_df["Frequency"] < (F_Q1 - 1.5 * F_IQR))].copy()
frequency_outliers_df.describe()


,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency
count,279.000000,279.000000,279.000000,279,279.000000
mean,15352.655914,14409.714932,23.810036,2010-11-23 11:06:20.645161216,16.089606
min,12437.000000,1094.390000,12.000000,2010-05-12 16:51:00,0.000000
25%,13800.000000,4331.560500,13.000000,2010-11-20 13:14:30,2.000000
50%,15465.000000,6615.770000,17.000000,2010-12-02 10:46:00,7.000000
75%,16828.500000,11692.405000,23.000000,2010-12-07 11:08:30,19.000000
max,18260.000000,349164.350000,183.000000,2010-12-09 19:32:00,211.000000
std,1748.429987,31381.736785,21.932937,NaN,26.589117


In [110]:
non_outliers_df = aggregated_df[(~aggregated_df.index.isin(monetary_outliers_df.index)) & (~aggregated_df.index.isin(frequency_outliers_df.index))]
non_outliers_df.describe()

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency
count,3809.000000,3809.000000,3809.000000,3809,3809.000000
mean,15376.482804,885.502197,2.863481,2010-09-03 11:16:46.516146176,97.082174
min,12346.000000,1.550000,1.000000,2009-12-01 10:49:00,0.000000
25%,13912.000000,279.910000,1.000000,2010-07-08 14:48:00,22.000000
50%,15389.000000,588.050000,2.000000,2010-10-12 16:25:00,58.000000
75%,16854.000000,1269.050000,4.000000,2010-11-17 13:14:00,154.000000
max,18287.000000,3788.210000,11.000000,2010-12-09 20:01:00,373.000000
std,1693.199806,817.672283,2.243748,NaN,98.111495


In [111]:
fig3 = make_subplots(rows=1, cols=3, subplot_titles=(
    "Monetary Value Boxplot",
    "Frequency Boxplot",
    "Recency Boxplot"
))

# Monetary Value
fig3.add_trace(
    go.Box(y=non_outliers_df["MonetaryValue"], boxpoints="outliers", marker_color="skyblue"),
    row=1, col=1
)

# Frequency
fig3.add_trace(
    go.Box(y=non_outliers_df["Frequency"], boxpoints="outliers", marker_color="lightgreen"),
    row=1, col=2
)

# Recency
fig3.add_trace(
    go.Box(y=non_outliers_df["Recency"], boxpoints="outliers", marker_color="salmon"),
    row=1, col=3
)

fig3.update_layout(height=400, width=1100, showlegend=False)
fig3

In [112]:
import plotly.express as px

fig4 = px.scatter_3d(
    non_outliers_df,
    x="MonetaryValue",
    y="Frequency",
    z="Recency",
    opacity=0.7,
    color="Recency",
    color_continuous_scale="Viridis",
    title="3D Scatter Plot of Customer Data"
)
fig4.update_traces(marker=dict(size=3))

fig4.update_layout(
    scene_camera=dict(
        eye=dict(x=1.8, y=1.8, z=1.2)
    ),
    scene=dict(
        xaxis_title="Monetary Value",
        yaxis_title="Frequency",
        zaxis_title="Recency"
    )
)
fig4

In [113]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(non_outliers_df[["MonetaryValue", "Frequency", "Recency"]])
scaled_data

array([[-0.87594534, -0.38488934,  0.68214853],
       [ 0.5355144 , -0.38488934, -0.96925093],
       [-0.81258645, -0.83063076, -0.24548944],
       ...,
       [-0.62197163, -0.83063076,  2.01753946],
       [ 0.44146683, -0.38488934,  0.14187587],
       [ 1.72488781,  0.50659348, -0.81634357]], shape=(3809, 3))

In [114]:
scaled_data_df = pd.DataFrame(scaled_data, index=non_outliers_df.index, columns=("MonetaryValue", "Frequency", "Recency"))
scaled_data_df

,MonetaryValue,Frequency,Recency
0,-0.875945,-0.384889,0.682149
1,0.535514,-0.384889,-0.969251
2,-0.812586,-0.830631,-0.245489
3,1.633678,-0.384889,-0.561498
4,-0.715016,-0.830631,-0.887700
...,...,...,...
4280,-0.298120,1.398076,-0.816344
4281,-0.579553,-0.830631,-0.316846
4282,-0.621972,-0.830631,2.017539
4283,0.441467,-0.384889,0.141876


In [115]:
fig5 = px.scatter_3d(
    scaled_data_df,
    x="MonetaryValue",
    y="Frequency",
    z="Recency",
    opacity=0.7,
    color="Recency",
    color_continuous_scale="Viridis",
    title="3D Scatter Plot of Customer Data"
)
fig5.update_traces(marker=dict(size=3))

fig5.update_layout(
    scene_camera=dict(
        eye=dict(x=1.8, y=1.8, z=1.2)
    ),
    scene=dict(
        xaxis_title="Monetary Value",
        yaxis_title="Frequency",
        zaxis_title="Recency"
    )
)
fig5

## 4. KMeans Clustering

In [117]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

max_k = 12
k_values = list(range(2, max_k + 1))
inertia = []
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=1000)
    labels = kmeans.fit_predict(scaled_data_df)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(scaled_data_df, labels))

fig6 = make_subplots(
    rows=1, cols=2,
    subplot_titles=("KMeans Inertia for Different Values of k",
                    "Silhouette Scores for Different Values of k")
)

# Inertia:
fig6.add_trace(
    go.Scatter(x=k_values, y=inertia, mode="lines+markers", name="Inertia"),
    row=1, col=1
)

# Silhouette:
fig6.add_trace(
    go.Scatter(x=k_values, y=silhouette_scores, mode="lines+markers",
               name="Silhouette", line=dict(color="orange")),
    row=1, col=2
)
fig6.update_xaxes(title_text="Number of Clusters (k)", tickmode="array", tickvals=k_values, row=1, col=2)
fig6.update_yaxes(title_text="Silhouette Score", row=1, col=2)

fig6.update_layout(height=450, width=1100, showlegend=False)
fig6

In [118]:
kmeans = KMeans(n_clusters=4, random_state=42, max_iter=1000)
cluster_labels = kmeans.fit_predict(scaled_data_df)
cluster_labels

array([1, 0, 2, ..., 1, 0, 0], shape=(3809,), dtype=int32)

In [119]:
non_outliers_df = non_outliers_df.copy()
non_outliers_df["Cluster"] = cluster_labels
non_outliers_df

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency,Cluster
0,12346.0,169.36,2,2010-06-28 13:53:00,164,1
1,12347.0,1323.32,2,2010-12-07 14:57:00,2,0
2,12348.0,221.16,1,2010-09-27 14:59:00,73,2
3,12349.0,2221.14,2,2010-10-28 08:23:00,42,0
4,12351.0,300.93,1,2010-11-29 15:23:00,10,2
...,...,...,...,...,...,...
4280,18283.0,641.77,6,2010-11-22 15:30:00,17,0
4281,18284.0,411.68,1,2010-10-04 11:33:00,66,2
4282,18285.0,377.00,1,2010-02-17 10:24:00,295,1
4283,18286.0,1246.43,2,2010-08-20 11:57:00,111,0


In [120]:
# Sicherstellen, dass Cluster als Kategorie behandelt wird
non_outliers_df = non_outliers_df.copy()
non_outliers_df["Cluster"] = non_outliers_df["Cluster"].astype("category")

# Deine Farbzuordnung
cluster_colors = {
    0: '#1f77b4',  # Blue
    1: '#ff7f0e',  # Orange
    2: '#2ca02c',  # Green
    3: '#d62728'   # Red
}

# Plotly: 3D-Scatter mit fester Farb-Map pro Cluster
fig7 = px.scatter_3d(
    non_outliers_df,
    x="MonetaryValue",
    y="Frequency",
    z="Recency",
    color="Cluster",
    color_discrete_map={str(k): v for k, v in cluster_colors.items()},  # Map auf Kategorienamen
    opacity=0.8,
    title="3D Scatter Plot of Customer Data by Cluster"
)

# Marker-Einstellungen (kleinere Punkte)
fig7.update_traces(marker=dict(size=2))

# Achsentitel + Kamera-Perspektive für gut lesbare Beschriftung
fig7.update_layout(
    scene=dict(
        xaxis_title="Monetary Value",
        yaxis_title="Frequency",
        zaxis_title="Recency"
    ),
    scene_camera=dict(eye=dict(x=1.8, y=1.8, z=1.2)),
    legend_title_text="Cluster"
)

fig7

In [121]:
# Subplot-Layout: 1 Reihe, 3 Spalten
fig8 = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Monetary Value by Cluster", "Frequency by Cluster", "Recency by Cluster")
)

# Monetary Value
fig8.add_trace(
    go.Violin(
        x=non_outliers_df["Cluster"],
        y=non_outliers_df["MonetaryValue"],
        box_visible=True,
        meanline_visible=True,
        points="all",
        marker=dict(size=2),
        name="Monetary Value"
    ),
    row=1, col=1
)

# Frequency
fig8.add_trace(
    go.Violin(
        x=non_outliers_df["Cluster"],
        y=non_outliers_df["Frequency"],
        box_visible=True,
        meanline_visible=True,
        points="all",
        marker=dict(size=2),
        name="Frequency"
    ),
    row=1, col=2
)

# Recency
fig8.add_trace(
    go.Violin(
        x=non_outliers_df["Cluster"],
        y=non_outliers_df["Recency"],
        box_visible=True,
        meanline_visible=True,
        points="all",
        marker=dict(size=2),
        name="Recency"
    ),
    row=1, col=3
)

# Layout-Anpassungen
fig8.update_layout(
    height=500, width=1200,
    showlegend=False,
    title_text="RFM-Verteilung nach Clustern"
)

fig8

In [122]:
overlap_indices = monetary_outliers_df.index.intersection(frequency_outliers_df.index)

monetary_only_outliers = monetary_outliers_df.drop(overlap_indices)
frequency_only_outliers = frequency_outliers_df.drop(overlap_indices)
monetary_and_frequency_outliers = monetary_outliers_df.loc[overlap_indices]

monetary_only_outliers["Cluster"] = -1
frequency_only_outliers["Cluster"] = -2
monetary_and_frequency_outliers["Cluster"] = -3

outlier_clusters_df = pd.concat([monetary_only_outliers, frequency_only_outliers, monetary_and_frequency_outliers])

outlier_clusters_df

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency,Cluster
9,12357.0,11229.99,1,2010-11-16 10:05:00,23,-1
25,12380.0,4782.84,4,2010-08-31 14:54:00,100,-1
42,12409.0,12346.62,4,2010-10-15 10:24:00,55,-1
48,12415.0,19468.84,4,2010-11-29 15:07:00,10,-1
61,12431.0,4145.52,11,2010-12-01 10:03:00,8,-1
...,...,...,...,...,...,...
4235,18223.0,7516.31,12,2010-11-17 12:20:00,22,-3
4236,18225.0,7545.14,15,2010-12-09 15:46:00,0,-3
4237,18226.0,6650.83,15,2010-11-26 15:51:00,13,-3
4241,18231.0,4791.80,23,2010-10-29 14:17:00,41,-3


In [123]:
# Farben für die Outlier-Cluster
cluster_colors = {-1: '#9467bd',  # Lila
                  -2: '#8c564b',  # Braun
                  -3: '#e377c2'}  # Rosa

fig9 = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Monetary Value by Outlier Cluster",
                    "Frequency by Outlier Cluster",
                    "Recency by Outlier Cluster")
)

# Monetary Value
for cluster, color in cluster_colors.items():
    fig9.add_trace(
        go.Violin(
            x=outlier_clusters_df[outlier_clusters_df["Cluster"] == cluster]["Cluster"],
            y=outlier_clusters_df[outlier_clusters_df["Cluster"] == cluster]["MonetaryValue"],
            name=f"Cluster {cluster}",
            box_visible=True,
            meanline_visible=True,
            line_color=color,
            fillcolor=color,
            opacity=0.6
        ),
        row=1, col=1
    )

# Frequency
for cluster, color in cluster_colors.items():
    fig9.add_trace(
        go.Violin(
            x=outlier_clusters_df[outlier_clusters_df["Cluster"] == cluster]["Cluster"],
            y=outlier_clusters_df[outlier_clusters_df["Cluster"] == cluster]["Frequency"],
            name=f"Cluster {cluster}",
            box_visible=True,
            meanline_visible=True,
            line_color=color,
            fillcolor=color,
            opacity=0.6
        ),
        row=1, col=2
    )

# Recency
for cluster, color in cluster_colors.items():
    fig9.add_trace(
        go.Violin(
            x=outlier_clusters_df[outlier_clusters_df["Cluster"] == cluster]["Cluster"],
            y=outlier_clusters_df[outlier_clusters_df["Cluster"] == cluster]["Recency"],
            name=f"Cluster {cluster}",
            box_visible=True,
            meanline_visible=True,
            line_color=color,
            fillcolor=color,
            opacity=0.6
        ),
        row=1, col=3
    )

fig9.update_layout(
    height=500, width=1200,
    title_text="RFM-Verteilung bei Ausreißer-Clustern",
    showlegend=True
)

fig9

In [124]:
cluster_labels = {
    0: "RETAIN",
    1: "RE-ENGAGE",
    2: "NURTURE",
    3: "REWARD",
    -1: "PAMPER",
    -2: "UPSELL",
    -3: "DELIGHT"
}

full_clustering_df = pd.concat([non_outliers_df, outlier_clusters_df])
full_clustering_df

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency,Cluster
0,12346.0,169.36,2,2010-06-28 13:53:00,164,1
1,12347.0,1323.32,2,2010-12-07 14:57:00,2,0
2,12348.0,221.16,1,2010-09-27 14:59:00,73,2
3,12349.0,2221.14,2,2010-10-28 08:23:00,42,0
4,12351.0,300.93,1,2010-11-29 15:23:00,10,2
...,...,...,...,...,...,...
4235,18223.0,7516.31,12,2010-11-17 12:20:00,22,-3
4236,18225.0,7545.14,15,2010-12-09 15:46:00,0,-3
4237,18226.0,6650.83,15,2010-11-26 15:51:00,13,-3
4241,18231.0,4791.80,23,2010-10-29 14:17:00,41,-3


In [125]:
full_clustering_df["ClusterLabel"] = full_clustering_df["Cluster"].map(cluster_labels)
full_clustering_df

,Customer ID,MonetaryValue,Frequency,lastInvoiceDate,Recency,Cluster,ClusterLabel
0,12346.0,169.36,2,2010-06-28 13:53:00,164,1,RE-ENGAGE
1,12347.0,1323.32,2,2010-12-07 14:57:00,2,0,RETAIN
2,12348.0,221.16,1,2010-09-27 14:59:00,73,2,NURTURE
3,12349.0,2221.14,2,2010-10-28 08:23:00,42,0,RETAIN
4,12351.0,300.93,1,2010-11-29 15:23:00,10,2,NURTURE
...,...,...,...,...,...,...,...
4235,18223.0,7516.31,12,2010-11-17 12:20:00,22,-3,DELIGHT
4236,18225.0,7545.14,15,2010-12-09 15:46:00,0,-3,DELIGHT
4237,18226.0,6650.83,15,2010-11-26 15:51:00,13,-3,DELIGHT
4241,18231.0,4791.80,23,2010-10-29 14:17:00,41,-3,DELIGHT


In [126]:
# 1) Größen je Cluster
cluster_counts = full_clustering_df['ClusterLabel'].value_counts()

# 2) Skaliertes Monetary-Feature
full_clustering_df["MonetaryValue per 100 pounds"] = (
    full_clustering_df["MonetaryValue"] / 100.0
)

# 3) Mittelwerte je Cluster
feature_means = (
    full_clustering_df
    .groupby('ClusterLabel')[['Recency', 'Frequency', 'MonetaryValue per 100 pounds']]
    .mean()
)

# 4) Einheitliche Reihenfolge festlegen (wie die Balken)
cat_order = list(cluster_counts.index)
feature_means = feature_means.reindex(cat_order)

# 5) Plot mit zwei y-Achsen
fig10 = make_subplots(specs=[[{"secondary_y": True}]])

# Balken: Anzahl Kunden pro Cluster
fig10.add_trace(
    go.Bar(
        x=cat_order,
        y=cluster_counts.reindex(cat_order).values,
        name="Number of Customers",
        marker_color="skyblue",
        opacity=0.8
    ),
    secondary_y=False
)

# Linien (Marker): durchschnittliche RFM-Werte – exakt in derselben Reihenfolge
fig10.add_trace(
    go.Scatter(
        x=cat_order, y=feature_means["Recency"].values,
        mode="lines+markers", name="Avg Recency", line=dict(color="#FFA500")
    ),
    secondary_y=True
)
fig10.add_trace(
    go.Scatter(
        x=cat_order, y=feature_means["Frequency"].values,
        mode="lines+markers", name="Avg Frequency", line=dict(color="#2ca02c")
    ),
    secondary_y=True
)
fig10.add_trace(
    go.Scatter(
        x=cat_order, y=feature_means["MonetaryValue per 100 pounds"].values,
        mode="lines+markers", name="Avg MonetaryValue per 100 pounds",
        line=dict(color="#9467bd")
    ),
    secondary_y=True
)

# Achsen & Layout
fig10.update_xaxes(
    title_text="Cluster",
    categoryorder="array",
    categoryarray=cat_order
)
fig10.update_yaxes(title_text="Number of Customers", secondary_y=False)
fig10.update_yaxes(title_text="Average Feature Values", secondary_y=True)

fig10.update_layout(
    title="Cluster Distribution with Average Feature Values",
    width=1000, height=600, template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0)
)

fig10